In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/carlo/workspace/networkqit/')
import numpy as np
import networkqit as nq
import networkx as nx
from numpy.random import random as rand
from networkqit import graph_laplacian as GL
from scipy.linalg import eigvalsh
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

Astar = np.loadtxt('/home/carlo/workspace/networkqit/notebooks/ring_3_4_5_6_7_8.adj',delimiter=',')
G = nx.from_numpy_array(Astar)
k = Astar.sum(axis=0).astype(int)
Lstar = nq.graph_laplacian(Astar)
pstar = nx.density(G)
N = len(G.nodes())
M=nq.UBCM(N=N)

optimizer = nq.MLEOptimizer(A=Astar,x0=np.random.random(N))
sol = optimizer.runfsolve(model='UBCM')

In [ ]:
beta_range = np.logspace(1,-3,100)

In [ ]:
# initialize from the solution
optimizer = nq.ExpectedModelOptimizer(A=Astar,beta_range=beta_range,x0=np.random.random(N))
optimizer.setup(expected_adj_fun=M,expected_lapl_grad_fun=M.expected_laplacian_grad,step_callback=lambda b,x:print('\r beta=%f'% b,end=''))
solspectral_bfgs_sol_reinit = optimizer.run(method='SLSQP', gtol=1E-3)

In [ ]:
pos = nx.spring_layout(G,iterations=25000,k=50)
nx.draw_networkx(G,pos=pos)

In [ ]:
allsol=np.loadtxt('allsol_ring_clique_ubcm_500.txt')
beta_range = np.logspace(2,-4,500)

In [ ]:
fig,ax = plt.subplots(ncols=5,nrows=70,squeeze=True,figsize=(25,25*14))
ax=ax.flatten()

for i,beta in enumerate(beta_range[0:350]):
    j=i
    Z=np.random.random([len(Astar),len(Astar)])
    P = M(allsol[:,i])
    Gsample = nx.from_numpy_array(P>Z)
    ax[j].cla()
    nx.draw_networkx_nodes(Gsample,pos=pos, node_size=50,ax=ax[j])
    nx.draw_networkx_edges(Gsample,pos=pos, ax=ax[j],edge_color='k',alpha=0.2)
    nx.draw_networkx_edges(G,      pos=pos, ax=ax[j],edge_color='r',alpha=0.9) # draw original edges
    ax[j].set_title('beta=%.3f deltaL=%.1f iter=%i' % (beta,(Astar.sum()-P.sum()),i))
    ax[j].axis('off')
plt.subplots_adjust()
fig.savefig('sampled_rc_3_4_5_6_7_8.png',dpi=100,bbox_inches='tight')

In [ ]:
np.arange(0,351,70)

In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=2,squeeze=True,figsize=(18,12))
ax=ax.flatten()

for j,i in enumerate(np.arange(0,351,70)):
    beta=beta_range[i]
    Z=np.random.random([len(Astar),len(Astar)])
    P = M(allsol[:,i])
    Gsample = nx.from_numpy_array(P>Z)
    ax[j].cla()
    nx.draw_networkx_nodes(Gsample,pos=pos, node_size=50,ax=ax[j])
    nx.draw_networkx_edges(Gsample,pos=pos, ax=ax[j],edge_color='k',alpha=0.2)
    #nx.draw_networkx_edges(G,      pos=pos, ax=ax[j],edge_color='r',alpha=0.9) # draw original edges
    ax[j].set_title('$\\beta$=%.3f  $\\Delta m$=%d' % (beta,(Astar.sum()-len(Gsample.edges()))))
    np.savetxt('sample_beta_%.2f' % (beta) + '.txt', nx.to_numpy_array(Gsample),delimiter=' ',fmt='%d')
    ax[j].axis('off')
plt.subplots_adjust()
fig.savefig('sampled_rc_3_4_5_6_7_8.png',dpi=100,bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(ncols=5,nrows=100,squeeze=True,figsize=(30,30*20))
ax=ax.flatten()

for i,beta in enumerate(beta_range):
    j=i
    Z=np.random.random([len(Astar),len(Astar)])
    P = M(allsol[:,i])
    Y = (P>Z).astype(float)
    ax[j].imshow(P)
    ax[j].grid(False)
    ax[j].set_title('beta=%.2f deltaL=%.1f iter=%i' % (beta,(Astar.sum()-P.sum()),i))
    ax[j].axis('off')
    
plt.subplots_adjust()

In [ ]:
Gsample = nx.from_numpy_array((M(allsol[:,5])>Z).astype(float))
pos = nx.spring_layout(Gsample,iterations=10000)
nx.draw_networkx(Gsample,pos)